In [1]:
import numpy as np
import pandas as pd
import mlflow
import os
import matplotlib.pyplot as plt
import matplotlib
import math
matplotlib.rcParams['figure.figsize'] = (12.0, 10.0)
from sklearn.metrics import ConfusionMatrixDisplay

from bokeh.plotting import show, output_file, output_notebook
import nntools.report as R
from datetime import datetime, timezone, timedelta

from mlflow.entities import ViewType

In [2]:
savepoint = 'http://localhost:5010'
exp_name = 'FundusSegmentation_MultiDatasets'
mlflow.set_tracking_uri(savepoint)

In [3]:
id_exp = mlflow.get_experiment_by_name(exp_name).experiment_id
runs = mlflow.search_runs(id_exp, run_view_type=ViewType.ALL)
runs = runs[(runs.status=='FINISHED') & (runs.start_time>datetime(2021, 8, 19, 23, tzinfo=timezone(-timedelta(hours=5))))]
runs['metrics.Test_mIoU_ddr']

0    0.507251
1    0.495369
2    0.510990
3    0.504798
4    0.477781
8    0.610288
Name: metrics.Test_mIoU_ddr, dtype: float64

In [4]:
runs['Train set'] = runs['tags.mlflow.runName'].map(lambda x: x.split('-')[0].replace('_', ' '))
columns = runs.columns
cols2drop = []
for c in columns:
    if c.startswith('params'):
        cols2drop.append(c)
runs = runs.drop(cols2drop, axis='columns')
METRICS = ['metrics.Test_mIoU_ddr', 'metrics.Test_mIoU_idrid', 'metrics.Test_mIoU_retinal_lesions']

LESIONS = ['Cotton_Wool_Spot', 'Exudates', 'Hemorrhages', 'Microaneurysms']

LESIONS_METRICS = ['metrics.%s_roc_%s'%(l, d) for l in LESIONS for d in ['idrid', 'ddr', 'retinal_lesions']]

df = runs[['Train set']+METRICS+LESIONS_METRICS]

In [5]:
df = df.sort_values(by='Train set')
old_cols = df.columns
new_cols = [c.replace('metrics.', '').replace('Test_', '').replace('_', ' ').replace('roc', 'AUROC') for c in old_cols]
df.columns = new_cols
df

,Train set,mIoU ddr,mIoU idrid,mIoU retinal lesions,Cotton Wool Spot AUROC idrid,Cotton Wool Spot AUROC ddr,Cotton Wool Spot AUROC retinal lesions,Exudates AUROC idrid,Exudates AUROC ddr,Exudates AUROC retinal lesions,Hemorrhages AUROC idrid,Hemorrhages AUROC ddr,Hemorrhages AUROC retinal lesions,Microaneurysms AUROC idrid,Microaneurysms AUROC ddr,Microaneurysms AUROC retinal lesions
3,FGADR,0.504798,0.540878,0.549951,0.559853,0.228508,0.228157,0.707796,0.462074,0.399941,0.545614,0.360680,0.302038,0.139456,0.069339,0.055669
8,IDRID,0.610288,0.704561,0.556553,0.720958,0.302653,0.167200,0.864346,0.496973,0.363586,0.689475,0.370854,0.173056,0.400159,0.210895,0.117029
1,IDRID FGADR,0.495369,0.509089,0.555192,0.705598,0.377482,0.285216,0.745269,0.343512,0.306361,0.657712,0.481463,0.311478,0.351837,0.072862,0.100813
4,IDRID MESSIDOR FGADR,0.477781,0.476371,0.537277,0.633032,0.331986,0.364244,0.751064,0.530888,0.436735,0.678952,0.440931,0.356097,0.311491,0.188706,0.086616
2,MESSIDOR,0.510990,0.504549,0.576113,0.533243,0.244502,0.277717,0.535313,0.381354,0.579299,0.509496,0.222896,0.249447,0.321354,0.249757,0.090247
0,MESSIDOR FGADR,0.507251,0.495235,0.569816,0.604116,0.341192,0.348380,0.696630,0.432293,0.401174,0.622702,0.399049,0.383797,0.299760,0.162553,0.094630


In [147]:
new_cols

['Train set',
 'mIoU_ddr',
 'mIoU_idrid',
 'mIoU_retinal_lesions',
 'Cotton_Wool_Spot_roc_idrid',
 'Cotton_Wool_Spot_roc_ddr',
 'Cotton_Wool_Spot_roc_retinal_lesions',
 'Exudates_roc_idrid',
 'Exudates_roc_ddr',
 'Exudates_roc_retinal_lesions',
 'Hemorrhages_roc_idrid',
 'Hemorrhages_roc_ddr',
 'Hemorrhages_roc_retinal_lesions',
 'Microaneurysms_roc_idrid',
 'Microaneurysms_roc_ddr',
 'Microaneurysms_roc_retinal_lesions']